In [31]:
import os
import shutil

import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow
tensorflow.__version__

'2.10.0'

# Data Preparation

The dataset contains around 1900 images of dinos and around 1900 images of dragons. 

The dataset contains separate folders for training and test sets. 

In [32]:
train_path = 'C:\\Users\\Daniel\\Downloads\\dino-dragon\\train'
valid_path = 'C:\\Users\\Daniel\\Downloads\\dino-dragon\\val'

# Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

In [33]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(learning_rate=0.002, momentum=0.8), metrics=['accuracy'])

# Question 1

Since we have a binary classification problem, what is the best loss function for us?

- `binary crossentropy` <-
- `focal loss`
- `mean squared error`
- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

# Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

- 9215873
- 11215873 <-
- 14215873
- 19215873

In [34]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 175232)            0         
                                                                 
 dense_8 (Dense)             (None, 64)                11214912  
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
__________________________________________

# Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [35]:
train_data_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = ImageDataGenerator(rescale=1./255)
train_gen = train_data_gen.flow_from_directory(train_path, target_size=(150, 150), batch_size=20, class_mode='binary', shuffle=True)
val_gen = val_data_gen.flow_from_directory(train_path, target_size=(150, 150), batch_size=20, class_mode='binary', shuffle=True)


Found 1594 images belonging to 2 classes.
Found 1594 images belonging to 2 classes.


In [36]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen)

Epoch 1/10
80/80 [==============================] - 28s 345ms/step - loss: 0.6187 - accuracy: 0.6556 - val_loss: 0.5310 - val_accuracy: 0.7051
Epoch 2/10
80/80 [==============================] - 27s 332ms/step - loss: 0.4748 - accuracy: 0.7911 - val_loss: 0.4200 - val_accuracy: 0.8394
Epoch 3/10
80/80 [==============================] - 27s 333ms/step - loss: 0.4025 - accuracy: 0.8356 - val_loss: 0.3456 - val_accuracy: 0.8695
Epoch 4/10
80/80 [==============================] - 27s 331ms/step - loss: 0.3616 - accuracy: 0.8532 - val_loss: 0.3637 - val_accuracy: 0.8482
Epoch 5/10
80/80 [==============================] - 27s 338ms/step - loss: 0.3205 - accuracy: 0.8827 - val_loss: 0.3847 - val_accuracy: 0.7949
Epoch 6/10
80/80 [==============================] - 36s 457ms/step - loss: 0.2719 - accuracy: 0.9003 - val_loss: 0.2350 - val_accuracy: 0.9191
Epoch 7/10
80/80 [==============================] - 35s 440ms/step - loss: 0.2396 - accuracy: 0.9103 - val_loss: 0.2312 - val_accuracy: 0.9122

# Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90 <-
- 0.20

In [37]:
accuracy = history.history['accuracy']
accuracy_median = np.median(accuracy)
accuracy_median

0.8914679884910583

# Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11 <-
- 0.66
- 0.99
- 0.33

In [39]:
loss = history.history['loss']
loss_std = np.std(loss)
loss_std

0.13443107654611655

# Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [40]:
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_gen = train_data_gen.flow_from_directory(train_path, target_size=(150, 150), batch_size=20, class_mode='binary', shuffle=True)
val_gen = val_data_gen.flow_from_directory(train_path, target_size=(150, 150), batch_size=20, class_mode='binary', shuffle=True)

Found 1594 images belonging to 2 classes.
Found 1594 images belonging to 2 classes.


# Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37 <-
- 0.97

In [41]:
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen)

Epoch 1/10
80/80 [==============================] - 48s 607ms/step - loss: 0.4430 - accuracy: 0.7974 - val_loss: 0.4232 - val_accuracy: 0.8080
Epoch 2/10
80/80 [==============================] - 43s 534ms/step - loss: 0.4163 - accuracy: 0.8124 - val_loss: 0.3718 - val_accuracy: 0.8425
Epoch 3/10
80/80 [==============================] - 42s 522ms/step - loss: 0.4061 - accuracy: 0.8124 - val_loss: 0.3761 - val_accuracy: 0.8394
Epoch 4/10
80/80 [==============================] - 41s 514ms/step - loss: 0.3730 - accuracy: 0.8381 - val_loss: 0.3734 - val_accuracy: 0.8325
Epoch 5/10
80/80 [==============================] - 41s 511ms/step - loss: 0.3791 - accuracy: 0.8237 - val_loss: 0.3366 - val_accuracy: 0.8726
Epoch 6/10
80/80 [==============================] - 41s 519ms/step - loss: 0.3427 - accuracy: 0.8488 - val_loss: 0.3164 - val_accuracy: 0.8821
Epoch 7/10
80/80 [==============================] - 46s 574ms/step - loss: 0.3651 - accuracy: 0.8513 - val_loss: 0.3594 - val_accuracy: 0.8400

In [44]:
loss = history.history['val_loss']
loss_mean = np.mean(loss)
loss_mean

0.34957538545131683

# Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84 <-
- 0.54
- 0.44
- 0.24

In [47]:
accuracy = history.history['accuracy'][5:10]
accuracy_mean = np.mean(accuracy)
accuracy_mean

0.849058985710144